In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.model_selection import train_test_split
#from sklearn import metrics
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

In [2]:
def complexity(ts):
    return np.sqrt(np.sum(np.diff(ts)**2))
# CID Distance function
def cid_distance(ts1, ts2):
    # Ensure both inputs are numpy arrays
    ts1 = np.asarray(ts1)
    ts2 = np.asarray(ts2)

    # Step 1: Compute standard Euclidean distance
    dist = euclidean(ts1, ts2)
    
    # Step 2: Compute the complexities of both time series
    c_ts1 = complexity(ts1)
    c_ts2 = complexity(ts2)
    
    # Step 3: Apply the CID formula
    return dist * np.sqrt(c_ts1 / c_ts2)

In [3]:
#Train with 10x10, test with full grid
data = pd.read_csv("../csv/tn/classif_fullgridmon_callin.csv") #to predict
data2 = pd.read_csv("../csv/19x19/classif_18x18_callin.csv") #daily

#dataa = pd.read_csv("../csv/tn/classif_fullgridmon_smsout.csv") #to predict
#dataa2 = pd.read_csv("../csv/15x15/classif_15x15_smsout.csv") #daily

dataaa = pd.read_csv("../csv/tn/classif_fullgridmon_callout.csv") #to predict
dataaa2 = pd.read_csv("../csv/19x19/classif_18x18_callout.csv") #daily

#data3 = pd.read_csv("../csv/randomgrid/classif-randomgrid_callout.csv") #daily
#data = pd.read_csv("../csv/20x20/classif20x20frid_callout.csv")

data2.head()
#20x20 grid, monday(4/11) - callout

,Unnamed: 0,cellid,callin0,callin1,callin2,callin3,callin4,callin5,callin6,callin7,...,callin15,callin16,callin17,callin18,callin19,callin20,callin21,callin22,callin23,WHF
0,0,4379,0.000204,0.000716,0.000118,0.0,0.000000,0.000324,0.000149,0.002093,...,0.009150,0.013073,0.010728,0.009591,0.008018,0.004955,0.003183,0.001443,0.000482,1
1,1,4380,0.001311,0.000462,0.000277,0.0,0.000101,0.000388,0.000334,0.004638,...,0.014708,0.017623,0.016723,0.013522,0.013662,0.008128,0.004873,0.002610,0.001061,2
2,2,4381,0.000364,0.000249,0.000000,0.0,0.000000,0.000218,0.000499,0.004747,...,0.010988,0.011554,0.011801,0.008018,0.007783,0.004720,0.002418,0.000904,0.000619,1
3,3,4382,0.001501,0.000915,0.000000,0.0,0.000000,0.001501,0.002771,0.010943,...,0.031121,0.046033,0.046125,0.048560,0.024682,0.020695,0.009119,0.004223,0.004496,1
4,4,4383,0.001647,0.000863,0.000014,0.0,0.000035,0.001550,0.002651,0.011641,...,0.033412,0.046762,0.046555,0.051328,0.025946,0.021297,0.010219,0.004508,0.005045,1


In [4]:
y = data2.iloc[:,26] #h/w
#y = y.append(data3.iloc[:,26])

x = data2.iloc[:,2:26] #sms, calls, callout
#x = x.append(data3.iloc[:,2:26])

#y2 = dataa2.iloc[:,26]
#x2 = dataa2.iloc[:,2:26]

y3 = dataaa2.iloc[:,26]
x3 = dataaa2.iloc[:,2:26]

x_test = data.iloc[:,1:26]
#x2_test = dataa.iloc[:,1:26]
x3_test = dataaa.iloc[:,1:26]

In [5]:
x_test

,callin0,callin1,callin2,callin3,callin4,callin5,callin6,callin7,callin8,callin9,...,callin14,callin15,callin16,callin17,callin18,callin19,callin20,callin21,callin22,callin23
0,0.000397,0.000092,0.000146,0.000073,0.000053,0.000017,0.000422,0.001953,0.006153,0.006898,...,0.005440,0.005237,0.006283,0.007518,0.005584,0.004321,0.002594,0.001025,0.000683,0.000185
1,0.000120,0.000131,0.000034,0.000020,0.000141,0.000065,0.000529,0.001953,0.005564,0.005086,...,0.004445,0.004652,0.005451,0.006118,0.006037,0.003009,0.002278,0.001406,0.000476,0.000120
2,0.000297,0.000178,0.000033,0.000053,0.000150,0.000074,0.000532,0.002583,0.004435,0.005690,...,0.003893,0.004977,0.005275,0.006546,0.006227,0.003803,0.002810,0.001506,0.000800,0.000567
3,0.000000,0.000000,0.000000,0.000000,0.000292,0.000240,0.000772,0.003491,0.008940,0.008845,...,0.007222,0.007785,0.008870,0.008676,0.011072,0.004839,0.004005,0.002911,0.000587,0.000240
4,0.000106,0.000000,0.000000,0.000000,0.000273,0.000150,0.000752,0.003194,0.007241,0.006992,...,0.005715,0.006243,0.007301,0.008448,0.009060,0.004263,0.003298,0.002405,0.000657,0.000213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6254,0.000000,0.000000,0.000000,0.000000,0.000002,0.000002,0.000000,0.000489,0.001992,0.001567,...,0.002023,0.001504,0.000889,0.001948,0.001664,0.001097,0.001293,0.000053,0.000019,0.000007
6255,0.000007,0.000000,0.000000,0.000000,0.000095,0.000139,0.000174,0.001184,0.002285,0.003488,...,0.003007,0.002739,0.002408,0.003367,0.002948,0.001729,0.001361,0.000527,0.000244,0.000041
6256,0.000040,0.000000,0.000194,0.000197,0.000194,0.000542,0.000891,0.006762,0.024761,0.036972,...,0.038861,0.037885,0.041505,0.035394,0.027053,0.016870,0.010753,0.006737,0.001953,0.000137
6257,0.000017,0.000000,0.000000,0.000103,0.000082,0.000355,0.000621,0.003070,0.009420,0.014301,...,0.015712,0.015252,0.015509,0.015085,0.010168,0.007157,0.003688,0.003197,0.001100,0.000139


In [6]:
#x_test.head()
#y

In [7]:
x

,callin0,callin1,callin2,callin3,callin4,callin5,callin6,callin7,callin8,callin9,...,callin14,callin15,callin16,callin17,callin18,callin19,callin20,callin21,callin22,callin23
0,0.000204,0.000716,0.000118,0.000000,0.000000,0.000324,0.000149,0.002093,0.010769,0.011923,...,0.010477,0.009150,0.013073,0.010728,0.009591,0.008018,0.004955,0.003183,0.001443,0.000482
1,0.001311,0.000462,0.000277,0.000000,0.000101,0.000388,0.000334,0.004638,0.017427,0.018757,...,0.019251,0.014708,0.017623,0.016723,0.013522,0.013662,0.008128,0.004873,0.002610,0.001061
2,0.000364,0.000249,0.000000,0.000000,0.000000,0.000218,0.000499,0.004747,0.008908,0.014253,...,0.011267,0.010988,0.011554,0.011801,0.008018,0.007783,0.004720,0.002418,0.000904,0.000619
3,0.001501,0.000915,0.000000,0.000000,0.000000,0.001501,0.002771,0.010943,0.027945,0.040825,...,0.030661,0.031121,0.046033,0.046125,0.048560,0.024682,0.020695,0.009119,0.004223,0.004496
4,0.001647,0.000863,0.000014,0.000000,0.000035,0.001550,0.002651,0.011641,0.031776,0.042637,...,0.032006,0.033412,0.046762,0.046555,0.051328,0.025946,0.021297,0.010219,0.004508,0.005045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,0.000165,0.000133,0.000050,0.000009,0.000004,0.000060,0.000441,0.002042,0.005742,0.009252,...,0.006211,0.005753,0.008601,0.008104,0.007851,0.005238,0.004285,0.002079,0.001038,0.000401
237,0.000004,0.000000,0.000000,0.000000,0.000162,0.000055,0.000181,0.001191,0.003764,0.005665,...,0.005421,0.007746,0.004931,0.004213,0.005871,0.003673,0.001501,0.001030,0.000457,0.000181
238,0.000015,0.000006,0.000000,0.000000,0.000147,0.000000,0.000270,0.001073,0.001949,0.003737,...,0.003056,0.005594,0.003316,0.002904,0.003742,0.002731,0.001231,0.000388,0.000242,0.000275
239,0.000091,0.000083,0.000000,0.000000,0.000008,0.000000,0.000102,0.000175,0.000318,0.000694,...,0.000661,0.000829,0.000319,0.000413,0.000579,0.000537,0.000322,0.000058,0.000024,0.000102


In [8]:
#knn = KNeighborsClassifier(n_neighbors=10, metric=cid_distance)
#knn2 = KNeighborsClassifier(n_neighbors=10)

rf1 = RandomForestClassifier(n_estimators=50, random_state=0)
rf2 = RandomForestClassifier(n_estimators=500, random_state=0)
rf3 = RandomForestClassifier(n_estimators=50, random_state=0)
#rf2 = RandomForestClassifier(n_estimators=50, random_state=0)
#rf3 = RandomForestClassifier(n_estimators=750, random_state=0)
#knn2 = rf

#knn.fit(x,y)
rf1.fit(x,y)
rf2.fit(x,y)

rf3.fit(x3,y3)
#rf2.fit(x2,y2)
#rf3.fit(x3,y3)

y_predt = rf1.predict(x_test)
y_predt2 = rf2.predict(x_test)
y_predt3 = rf3.predict(x3_test)

#y_predt2 = rf2.predict(x2_test) #predict h/w for all the grid (100x100)
#y_predt3 = rf3.predict(x3_test) #predict h/w for all the grid (100x100)


In [9]:
#y_predt = rf.predict(x_test) #predict h/w for all the grid (100x100)

#model = KMeans(n_clusters=5)
#model = AffinityPropagation(random_state=5) - doesn't converge
#yhat=model.fit_predict(x_test)

In [10]:
predicted = pd.DataFrame(np.transpose(y_predt),columns=['predicted'])
predicted['cellid'] = data.cellid
predicted.head() #callin50
#Predictions for the whole grid using callout data from the 20x20 grid

,predicted,cellid
0,2,38
1,2,39
2,2,40
3,2,154
4,2,155


In [11]:
predicted.to_csv('../csv/pred_fullgrid_trento_callin_rf50_19.csv')

In [12]:
predicted2 = pd.DataFrame(np.transpose(y_predt2),columns=['predicted'])
predicted2['cellid'] = data.cellid
predicted2.head()
#callin500

,predicted,cellid
0,2,38
1,2,39
2,2,40
3,2,154
4,2,155


In [13]:
predicted2.to_csv('../csv/pred_fullgrid_trento_callin_rf500_19.csv')

In [14]:
predicted3 = pd.DataFrame(np.transpose(y_predt3),columns=['predicted'])
predicted3['cellid'] = data.cellid
predicted3.head() #callout50

,predicted,cellid
0,2,38
1,1,39
2,2,40
3,2,154
4,2,155


In [15]:
predicted3.to_csv('../csv/pred_fullgrid_trento_callout_rf50_19.csv')